In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qutip import *
from scipy.integrate import quad

# Constants
hbar = 1.0
kB = 1.0

# System parameters
omega_0 = 1.0        # TLS energy gap
theta = np.pi / 4    # Mixing angle: 0 = transverse, pi/2 = dephasing

# Bath parameters
gamma = 0.1          # Damping
Omega = 5.0          # Cutoff frequency
T = 1.0              # Temperature (in units where kB = 1)

# Time grid
tlist = np.linspace(0, 20, 200)

# Operators
sz = sigmaz()
sx = sigmax()
identity = qeye(2)
H_sys = 0.5 * omega_0 * sz
initial_state = basis(2, 0) * basis(2, 0).dag()  # |0⟩⟨0|

# Spectral density (Ohmic with Lorentz-Drude cutoff)
def J(omega):
    return 2 * gamma * omega * Omega**2 / (np.pi * (omega**2 + Omega**2))

# Bath correlation function C(t)
def C_bath(t):
    integrand = lambda w: J(w) * (1/np.tanh(w / (2*T))) * np.cos(w*t)
    real_part, _ = quad(integrand, 0, np.inf, limit=200)
    return real_part

# Memory kernel: ξ(ω, t) = ∫₀ᵗ ds e^{iωs} C(s)
def xi(omega, t):
    integrand_real = lambda s: C_bath(s) * np.cos(omega*s)
    integrand_imag = lambda s: C_bath(s) * np.sin(omega*s)
    real, _ = quad(integrand_real, 0, t, limit=200)
    imag, _ = quad(integrand_imag, 0, t, limit=200)
    return real + 1j * imag

# Time-dependent Liouvillian construction
def L_t(t, rho):
    ξ_0 = xi(0, t)
    ξ_w = xi(omega_0, t)
    ξ_mw = xi(-omega_0, t)

    # Gamma coefficients
    gamma_pp = 0.5 * np.cos(theta)**2 * np.real(ξ_mw)
    gamma_mm = 0.5 * np.cos(theta)**2 * np.real(ξ_w)
    gamma_pm = 0.25 * np.cos(theta)**2 * (ξ_mw + np.conj(ξ_w)).real
    gamma_zp = 0.25 * np.sin(theta) * np.cos(theta) * (ξ_0 + np.conj(ξ_mw)).real
    gamma_zm = 0.25 * np.sin(theta) * np.cos(theta) * (ξ_0 + np.conj(ξ_w)).real

    # Collapse-like terms (not actual collapse ops — TCL form)
    L = (
        gamma_pp * (2 * sigmap() * rho * sigmam() - sigmam() * sigmap() * rho - rho * sigmam() * sigmap()) +
        gamma_mm * (2 * sigmam() * rho * sigmap() - sigmap() * sigmam() * rho - rho * sigmap() * sigmam())
    )
    return L

# Solve the master equation using Euler integration (since TCL is not built-in)
dt = tlist[1] - tlist[0]
rho_t = initial_state
rho_list = [rho_t]
spin_current = []
heat_current = []

for t in tlist[:-1]:
    L_rho = L_t(t, rho_t)
    drho_dt = -1j * (H_sys * rho_t - rho_t * H_sys) + L_rho
    rho_t = rho_t + dt * drho_dt
    rho_list.append(rho_t)

    # Spin current: d⟨σ_z⟩/dt
    dsz_dt = expect(sz, rho_t + dt * drho_dt) - expect(sz, rho_t)
    spin_current.append(dsz_dt / dt)

    # Heat current: Tr[H_sys * L_t(rho)]
    J_Q = (H_sys * L_rho).tr().real
    heat_current.append(J_Q)

spin_current.append(spin_current[-1])  # match length
heat_current.append(heat_current[-1])

# Plotting
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(tlist, spin_current, label='Spin Current')
plt.xlabel("Time")
plt.ylabel("I_s(t)")
plt.grid()
plt.legend()

plt.subplot(1,2,2)
plt.plot(tlist, heat_current, label='Heat Current', color='orange')
plt.xlabel("Time")
plt.ylabel("J_Q(t)")
plt.grid()
plt.legend()

plt.tight_layout()
plt.show()


/tmp/ipykernel_5722/3206899103.py:36: IntegrationWarning: The maximum number of subdivisions (200) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  real_part, _ = quad(integrand, 0, np.inf, limit=200)
/tmp/ipykernel_5722/3206899103.py:36: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  real_part, _ = quad(integrand, 0, np.inf, limit=200)
/tmp/ipykernel_5722/3206899103.py:36: IntegrationWarning: The algorithm does not converge.  Roundoff error is detected
  in the extrapolation table.  It is assumed that the requested tolerance
  cannot be achieved, and that the returned result (if full_output = 1) is 
  the bes